<a href="https://colab.research.google.com/github/seeun7/face/blob/main/%EC%84%A0%EB%8C%80_%EC%9B%B9%EC%BA%A0_%EC%97%B4%EA%B8%B0%2C_detection_box.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np

# 웹캠 열기
cap = cv2.VideoCapture(0)

# 얼굴 인식용 Haar Cascade 로드
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# 가정: 매칭된 얼굴의 인코딩과 이름 리스트
# 인코딩 예시는 실사용 시 실제 데이터로 대체해야 함
known_face_encodings = [
    np.array([0.1, 0.2, 0.3]),  # 예시 인코딩 벡터
    np.array([0.4, 0.5, 0.6])
]
known_face_names = [
    "Person1",
    "Person2"
]

def compare_faces(encoding, known_encodings, threshold=0.5):
    # 인코딩 간의 유사성을 계산하는 함수 (유클리디안 거리 사용)
    matches = []
    for known_encoding in known_encodings:
        similarity = np.linalg.norm(encoding - known_encoding)  # 유클리디안 거리 계산
        matches.append(similarity <= threshold)  # 유사도가 threshold 이하이면 매칭으로 간주
    return matches

def get_face_encoding(face_roi):
    # 얼굴 인코딩 계산을 위한 가상의 함수
    # 실제로는 얼굴 인코딩을 계산하는 로직을 구현해야 함
    # 여기는 임의의 인코딩 벡터를 반환하도록 설정
    return np.array([0.1, 0.2, 0.3])  # 예시 인코딩 벡터

while True:
    # 프레임 읽기
    ret, frame = cap.read()
    if not ret:
        break

    # 그레이스케일로 변환
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # 얼굴 검출
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # 검출된 각 얼굴에 대해
    for (x, y, w, h) in faces:
        # 얼굴 부분을 잘라서 인코딩 계산
        face_roi = frame[y:y+h, x:x+w]
        face_encoding = get_face_encoding(face_roi)  # 얼굴 인코딩 계산

        # 얼굴 비교
        matches = compare_faces(face_encoding, known_face_encodings)
        name = "저장되지 않은 얼굴입니다."

        # 매칭된 얼굴이 있는 경우
        if True in matches:
            first_match_index = matches.index(True)
            name = known_face_names[first_match_index]

        # 얼굴에 사각형과 이름 표시
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        cv2.putText(frame, name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

    # 결과 화면 표시
    cv2.imshow('Face Detection', frame)

    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 리소스 해제
cap.release()
cv2.destroyAllWindows()